### Implementação RNN

#### Bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import enchant
import torch
import torch.nn as nn


#nltk.download('punkt')   # for first-time use only. Punkt is a Sentence Tokenizer
#nltk.download('wordnet')    # for first-time use only. WordNet is a large lexical database of English.
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')

#### Importando o dataset

In [ ]:
# Importando o Dataset
text = pd.read_csv('chennai_reviews.csv', sep=',')
text.head()

#### Retirando colunas desnecessárias

In [ ]:
text = text.drop(['Hotel_name', 'Review_Title','Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',], axis=1)
text.head()

#### Atribuindo as variáveis review e sentiment

In [ ]:
review = text.Review_Text
sentiment = text.Sentiment

#### Remove todos os sentiments cujo tamanho da string é maior que 1

In [ ]:
for i in sentiment.index:
    if len(str(sentiment[i])) > 1:
        sentiment.drop(labels=i, inplace=True)
        review.drop(labels=i, inplace=True)

#### Filtrar as reviews: remover stop words e utilizar somente palavras necessárias para o entendimento da sentença

In [ ]:
review_tag = []
dictionary_en_US = enchant.Dict("en_US")

palavras = []

for r in review:
    tag_list = []
    tokens = nltk.word_tokenize(str(r))
    #split_r = str(r).split()
    #palavras = set(split_r)
    english_stop_words = set(stopwords.words('english'))
    palavras_filtradas = set(tokens) - english_stop_words
    palavras_filtradas = set(filter(dictionary_en_US.check, palavras_filtradas))
    tagged = nltk.pos_tag(palavras_filtradas)
    
    for t in tagged:
        if(t[1] == 'VERB' or t[1] == 'RB' or t[1] == 'JJ' or t[1] == 'RBR'):
            tag_list.append(t[0])
            palavras.append(t[0])
            
    review_tag.append(tag_list)
    

#### Remover linhas cujas reviews estão vazias

In [ ]:
sentiment1 = []
review_tag1 = []

for i in range(len(review_tag)):
    if len(review_tag[i]) != 0:
        sentiment1.append(list(sentiment)[i])
        review_tag1.append(review_tag[i])

#### Identifica as palavras únicas nas reviews

In [ ]:
all_words = np.unique(palavras)
len(np.unique(palavras))

#### Funções para preparar os dados

- wordToIndex: para utilizar o index de cada palavra
- wordToTensor: converter as palavras em tensorer
- lineToTensor: converte cada sentença em um tensor

In [ ]:
# Find word index from all_letters, e.g. "a" = 0
def wordToIndex(word):
    return np.where(all_words == word)[0][0]

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def wordToTensor(word):
    tensor = torch.zeros(1, len(all_words))
    tensor[0][wordToIndex(word)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, len(all_words))
    for li, word in enumerate(line):
        tensor[li][0][wordToIndex(word)] = 1
    return tensor

#### Classe RNN

In [ ]:
n_categories = 3 #Quantidade de classes para classificar
n_hidden = 128
feelings = np.array([1,2,3])

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(len(all_words), n_hidden, n_categories)

#### Para otimizar o programa, usa-se LineToTensor para criar um novo tensor a cada sentença

In [ ]:
input = lineToTensor(review_tag[1])
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

#### Função para interpetrar a saída da rede, que vai indicar a verosemelhança (likelihood) de cada categoria

In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return feelings[category_i], category_i

print(categoryFromOutput(output))

#### Exemplo de treinamento

In [ ]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(feelings)
    position = randomChoice(np.where(np.array(y_train) == str(category))[0])
    line = x_train[position]
    
    category_tensor = torch.tensor([np.where(feelings == category)[0]][0], dtype=torch.long)
    
    line_tensor = lineToTensor(line)
    
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

#### Separa os dados em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(review_tag1, sentiment1, test_size=0.33)

#### Função de treinamento

In [ ]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()
    
    #print(line_tensor.size()[0])

    for i in range(line_tensor.size()[0]):
        output1, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output1, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

#### Execução do programa: a cada 100 iterações, exibe a perda e verifica se a categoria corresponde à saída do review. Se sim, escreve o símbolo '✓'. Se não, escreve o símbolo '✗' e indica a categoria correta

In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000

criterion = nn.NLLLoss()

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

#### Plotar o gráfico das perdas

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

#### Exibir matriz de confusão

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = np.where(feelings == category)[0][0]
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + feelings, rotation=90)
ax.set_yticklabels([''] + feelings)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

#### Plotar predição

In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, feelings[category_index]))
            predictions.append([value, feelings[category_index]])

for i in range(5):
    predict(x_test[i])